In [1]:
!wget https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/nasa/nasa_19950731.22-19950831.22.tsv

--2020-04-07 21:23:15--  https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/nasa/nasa_19950731.22-19950831.22.tsv
Resolving hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)... 52.218.218.169
Connecting to hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)|52.218.218.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120704766 (115M) [application/x-www-form-urlencoded]
Saving to: ‘nasa_19950731.22-19950831.22.tsv’

nasa_19950731.22-19 100%[===================>] 115.11M  5.60MB/s    in 16s     

2020-04-07 21:23:31 (7.31 MB/s) - ‘nasa_19950731.22-19950831.22.tsv’ saved [120704766/120704766]



In [2]:
!wget https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/nasa/nasa_19950630.22-19950728.12.tsv

--2020-04-07 21:23:32--  https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/nasa/nasa_19950630.22-19950728.12.tsv
Resolving hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)... 52.218.193.105
Connecting to hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)|52.218.193.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148483808 (142M) [application/x-www-form-urlencoded]
Saving to: ‘nasa_19950630.22-19950728.12.tsv’

nasa_19950630.22-19 100%[===================>] 141.60M  7.52MB/s    in 49s     

2020-04-07 21:24:21 (2.89 MB/s) - ‘nasa_19950630.22-19950728.12.tsv’ saved [148483808/148483808]



In [3]:
!mv nasa_* ../data/

In [4]:
!rm -rf ../data/NASA

In [5]:
!ls ../data

nasa_19950630.22-19950728.12.tsv  nasa_19950731.22-19950831.22.tsv


In [6]:
from pyspark import sql

spark = sql.SparkSession.builder \
        .master("local[8]") \
        .getOrCreate()


In [7]:
log_1_path = "../data/nasa_19950630.22-19950728.12.tsv"
nasa_log_1_df = spark.read.csv(log_1_path)
nasa_log_1_df.show()


+--------------------+
|                 _c0|
+--------------------+
|host	logname	time...|
|199.72.81.55	-	80...|
|unicomp6.unicomp....|
|199.120.110.21	-	...|
|burger.letters.co...|
|199.120.110.21	-	...|
|burger.letters.co...|
|burger.letters.co...|
|205.212.115.106	-...|
|d104.aa.net	-	804...|
|129.94.144.152	-	...|
|unicomp6.unicomp....|
|unicomp6.unicomp....|
|unicomp6.unicomp....|
|d104.aa.net	-	804...|
|d104.aa.net	-	804...|
|d104.aa.net	-	804...|
|129.94.144.152	-	...|
|199.120.110.21	-	...|
|ppptky391.asahi-n...|
+--------------------+
only showing top 20 rows



In [8]:
nasa_log_1_df = (spark.read
                 .option("header", True)
                 .csv(log_1_path))

In [9]:
nasa_log_1_df = (spark.read
                 .option("header", True)
                 .option("sep", "\t")
                 .csv(log_1_path))
nasa_log_1_df.show()


+--------------------+-------+---------+------+--------------------+--------+-----+-------+---------+
|                host|logname|     time|method|                 url|response|bytes|referer|useragent|
+--------------------+-------+---------+------+--------------------+--------+-----+-------+---------+
|        199.72.81.55|      -|804571201|   GET|    /history/apollo/|     200| 6245|   null|     null|
|unicomp6.unicomp.net|      -|804571206|   GET| /shuttle/countdown/|     200| 3985|   null|     null|
|      199.120.110.21|      -|804571209|   GET|/shuttle/missions...|     200| 4085|   null|     null|
|  burger.letters.com|      -|804571211|   GET|/shuttle/countdow...|     304|    0|   null|     null|
|      199.120.110.21|      -|804571211|   GET|/shuttle/missions...|     200| 4179|   null|     null|
|  burger.letters.com|      -|804571212|   GET|/images/NASA-logo...|     304|    0|   null|     null|
|  burger.letters.com|      -|804571212|   GET|/shuttle/countdow...|     200|    0

In [10]:
nasa_log_1_df.count()

1891709

In [11]:
import sys

nasa_sample_df = nasa_log_1_df.limit(500).toPandas()
sys.getsizeof(nasa_sample_df), sys.getsizeof(nasa_log_1_df)


(259024, 64)

In [12]:
from pyspark.sql.functions import col, from_unixtime, unix_timestamp

nasa_log_1_parse_time_df = (nasa_log_1_df
    .withColumn("unixtime", from_unixtime(col("time")))
    .withColumn("timestamp", unix_timestamp(col("unixtime")).cast("timestamp"))
    .withColumn("bytes", col("bytes").cast("integer"))
    .withColumn("response", col("response").cast("integer")))

nasa_log_1_parse_time_df.show()


+--------------------+-------+---------+------+--------------------+--------+-----+-------+---------+-------------------+-------------------+
|                host|logname|     time|method|                 url|response|bytes|referer|useragent|           unixtime|          timestamp|
+--------------------+-------+---------+------+--------------------+--------+-----+-------+---------+-------------------+-------------------+
|        199.72.81.55|      -|804571201|   GET|    /history/apollo/|     200| 6245|   null|     null|1995-07-01 04:00:01|1995-07-01 04:00:01|
|unicomp6.unicomp.net|      -|804571206|   GET| /shuttle/countdown/|     200| 3985|   null|     null|1995-07-01 04:00:06|1995-07-01 04:00:06|
|      199.120.110.21|      -|804571209|   GET|/shuttle/missions...|     200| 4085|   null|     null|1995-07-01 04:00:09|1995-07-01 04:00:09|
|  burger.letters.com|      -|804571211|   GET|/shuttle/countdow...|     304|    0|   null|     null|1995-07-01 04:00:11|1995-07-01 04:00:11|
|     

In [13]:
from pyspark.sql.functions import col, hour, from_utc_timestamp

getRequestsDF = (nasa_log_1_parse_time_df
  .filter((col("response") >= 200) & (col("response") < 300))
  .select("method", "url", "timestamp", hour("timestamp").alias("hour"))
)
getRequestsDF.show()

+------+--------------------+-------------------+----+
|method|                 url|          timestamp|hour|
+------+--------------------+-------------------+----+
|   GET|    /history/apollo/|1995-07-01 04:00:01|   4|
|   GET| /shuttle/countdown/|1995-07-01 04:00:06|   4|
|   GET|/shuttle/missions...|1995-07-01 04:00:09|   4|
|   GET|/shuttle/missions...|1995-07-01 04:00:11|   4|
|   GET|/shuttle/countdow...|1995-07-01 04:00:12|   4|
|   GET|/shuttle/countdow...|1995-07-01 04:00:12|   4|
|   GET| /shuttle/countdown/|1995-07-01 04:00:13|   4|
|   GET|                   /|1995-07-01 04:00:13|   4|
|   GET|/shuttle/countdow...|1995-07-01 04:00:14|   4|
|   GET|/images/NASA-logo...|1995-07-01 04:00:14|   4|
|   GET|/images/KSC-logos...|1995-07-01 04:00:14|   4|
|   GET|/shuttle/countdow...|1995-07-01 04:00:15|   4|
|   GET|/images/NASA-logo...|1995-07-01 04:00:15|   4|
|   GET|/images/KSC-logos...|1995-07-01 04:00:15|   4|
|   GET|/images/launch-lo...|1995-07-01 04:00:17|   4|
|   GET|/f

In [14]:
getRequestsDF.groupby("hour").count().show()

+----+------+
|hour| count|
+----+------+
|  12| 74063|
|  22| 71734|
|   1| 65401|
|  13| 88009|
|   6| 40737|
|  16|109035|
|   3| 63327|
|  20|106904|
|   5| 48181|
|  19|109957|
|  15|103238|
|   9| 28538|
|  17|108794|
|   4| 56587|
|   8| 28910|
|  23| 65103|
|   7| 33817|
|  10| 31497|
|  21| 88520|
|  11| 48064|
+----+------+
only showing top 20 rows



In [15]:
(getRequestsDF
        .groupby("url")
        .count()
        .orderBy(col("count").desc())
        .show())

+--------------------+-----+
|                 url|count|
+--------------------+-----+
|/images/NASA-logo...|90378|
|/images/KSC-logos...|77204|
|/images/MOSAIC-lo...|53840|
|/images/USA-logos...|53437|
|/images/WORLD-log...|53076|
|/images/ksclogo-m...|52965|
|           /ksc.html|38606|
| /shuttle/countdown/|36934|
|/images/launch-lo...|36243|
|/images/ksclogosm...|30494|
|                   /|30218|
|/history/apollo/i...|27938|
|/shuttle/missions...|23257|
|  /htbin/cdt_main.pl|22626|
|/shuttle/countdow...|20643|
|/images/launchmed...|19385|
|/shuttle/countdow...|19352|
|/shuttle/countdow...|18491|
|/shuttle/missions...|18197|
|/shuttle/missions...|16675|
+--------------------+-----+
only showing top 20 rows



In [16]:
nasa_log_1_df.write.mode("overwrite").parquet("../data/NASA")

In [17]:
!ls ../data/NASA

part-00000-b907fc9c-aa40-46e4-ab46-bf23af6d5427-c000.snappy.parquet
part-00001-b907fc9c-aa40-46e4-ab46-bf23af6d5427-c000.snappy.parquet
part-00002-b907fc9c-aa40-46e4-ab46-bf23af6d5427-c000.snappy.parquet
part-00003-b907fc9c-aa40-46e4-ab46-bf23af6d5427-c000.snappy.parquet
part-00004-b907fc9c-aa40-46e4-ab46-bf23af6d5427-c000.snappy.parquet
part-00005-b907fc9c-aa40-46e4-ab46-bf23af6d5427-c000.snappy.parquet
part-00006-b907fc9c-aa40-46e4-ab46-bf23af6d5427-c000.snappy.parquet
part-00007-b907fc9c-aa40-46e4-ab46-bf23af6d5427-c000.snappy.parquet
_SUCCESS


In [18]:
spark.read.parquet("../data/NASA")

DataFrame[host: string, logname: string, time: string, method: string, url: string, response: string, bytes: string, referer: string, useragent: string]

In [19]:
spark.read.parquet("../data/NASA").count()

1891709

# Exercises

### 1

In [20]:
(getRequestsDF
        .groupby("url")
        .count()
        .orderBy(col("count").desc())
        .limit(10)
        .show())

+--------------------+-----+
|                 url|count|
+--------------------+-----+
|/images/NASA-logo...|90378|
|/images/KSC-logos...|77204|
|/images/MOSAIC-lo...|53840|
|/images/USA-logos...|53437|
|/images/WORLD-log...|53076|
|/images/ksclogo-m...|52965|
|           /ksc.html|38606|
| /shuttle/countdown/|36934|
|/images/launch-lo...|36243|
|/images/ksclogosm...|30494|
+--------------------+-----+



### 2

In [21]:
(getRequestsDF
        .groupby("url")
        .count()
        .orderBy(col("count").desc())
        .limit(10)
        .write.mode("overwrite").parquet("../data/NASA-top-10"))

### 3

In [22]:
log_2_path = "../data/nasa_19950731.22-19950831.22.tsv"
nasa_log_2_df = (spark.read
                 .option("header", True)
                 .option("sep", "\t").csv(log_2_path))

nasa_log_2_parse_time_df = (nasa_log_2_df
    .withColumn("unixtime", from_unixtime(col("time")))
    .withColumn("timestamp", unix_timestamp(col("unixtime")).cast("timestamp"))
    .withColumn("bytes", col("bytes").cast("integer"))
    .withColumn("response", col("response").cast("integer")))

nasa_log_2_df.write.mode("append").parquet("../data/NASA")

spark.read.parquet("../data/NASA").count()

3461595